<a href="https://colab.research.google.com/github/AdamJuma133/Crop-Plant-Disease-Identifier-Project/blob/main/Copy_of_Plant_diseases_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
abdallahalidev_plantvillage_dataset_path = kagglehub.dataset_download('abdallahalidev/plantvillage-dataset')

print('Data source import complete.')

In [ ]:
#importing the necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import image_dataset_from_directory
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16

In [ ]:
# applying different data augumentation methods to the training data
train_gen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 30,
    shear_range = 0.2,
    zoom_range = 0.2,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip = True,
    fill_mode = 'nearest',
    validation_split = 0.2
)

valid_gen = ImageDataGenerator(
    rescale = 1/255,
    validation_split=0.2
)

In [ ]:
Image_size = (255, 255)
Batch_size = 32

In [ ]:
#Loading the training and validation data from the director
data_dir = "../input/plantvillage-dataset/color"
train_generator = train_gen.flow_from_directory(
    data_dir,
    target_size = Image_size,
    batch_size = Batch_size,
    subset= "training",
    class_mode = "categorical",
    color_mode = "rgb"

)

valid_generator = valid_gen.flow_from_directory(
    data_dir,
    target_size = Image_size,
    batch_size = Batch_size,
    subset= "validation",
    class_mode = "categorical",
    color_mode = "rgb"


)

In [ ]:
model = VGG16(include_top=False, input_shape=(255,255,3), weights="imagenet")
for layer in model.layers:
    layer.trainable = False
x = Flatten()(model.output)
dense1 = Dense(1024, activation="relu")(x)
prediction = Dense(38, activation="softmax")(dense1)
model = Model(inputs=model.inputs, outputs=prediction)

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics  = ['accuracy']
    )

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.n // train_generator.batch_size,    #The 2 slashes division return rounded integer
        epochs = 5,
        validation_data = valid_generator,
        validation_steps = valid_generator.n // valid_generator.batch_size
        )

In [ ]:
model.save('final1_model.h5')